![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/TRANSLATION_PIPELINES_MODELS.ipynb)

# Spark NLP Translation Models & Pipelines

## 0. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.1

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

In [ ]:
import json
import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

## **Pipelines**: Open-Source Marian Translation Pipelines

### Translate English to Hausa Pipeline

In [ ]:
pipeline = PretrainedPipeline("translate_en_ha", lang = "xx") 
pipeline.annotate("Your sentence to translate!")

translate_en_ha download started this may take some time.
Approx size to download 267.8 MB
[OK!]


{'document': ['Your sentence to translate!'],
 'sentence': ['Your sentence to translate!'],
 'translation': ['Hukuncinka na fassara!']}

### Translate English to Azerbaijani Pipeline

In [ ]:
pipeline = PretrainedPipeline("translate_en_az", lang = "xx") 
pipeline.annotate("Your sentence to translate!")

translate_en_az download started this may take some time.
Approx size to download 203.7 MB
[OK!]


{'document': ['Your sentence to translate!'],
 'sentence': ['Your sentence to translate!'],
 'translation': ['Sənin tərcüməni tərcümə etmək üçün Sən öz dilinlə!']}

### Translate English to Baltic languages Pipeline

In [ ]:
pipeline = PretrainedPipeline("translate_en_bat", lang = "xx") 
pipeline.annotate("Your sentence to translate!")

translate_en_bat download started this may take some time.
Approx size to download 269.9 MB
[OK!]


{'document': ['Your sentence to translate!'],
 'sentence': ['Your sentence to translate!'],
 'translation': ['Tavas teikimas tulkot!']}

### Translate English to Arabic Pipeline

In [ ]:
pipeline = PretrainedPipeline("translate_en_ar", lang = "xx") 
pipeline.annotate("Your sentence to translate!")

translate_en_ar download started this may take some time.
Approx size to download 276.9 MB
[OK!]


{'document': ['Your sentence to translate!'],
 'sentence': ['Your sentence to translate!'],
 'translation': ['! عقوبتك لتترجم']}

## **Models**: Open-Source Marian Fast Neural Machine Translation Models

### Translation Model from Spanish to Waray

In [ ]:
input_dict = {
1: "Además de ser el rey del norte, John Snow es un médico inglés y líder en el desarrollo de la anestesia y la higiene médica. Se le considera el primero en utilizar datos para curar el brote de cólera en 1854.",
2: "Titanic es una película épica estadounidense de 1997 sobre desastres y romance dirigida, escrita, coproducida y coeditada por James Cameron. Incorpora aspectos históricos y ficticios, y se basa en relatos del hundimiento del RMS Titanic. Está protagonizada por Leonardo DiCaprio y Kate Winslet como miembros de diferentes clases sociales que se enamoran a bordo del barco durante su desafortunado viaje inaugural.",
3: "William Henry Gates III (nacido el 28 de octubre de 1955) es un magnate empresarial, desarrollador de software, inversor y filántropo estadounidense. Es mejor conocido como cofundador de Microsoft Corporation. Durante su carrera en Microsoft, Gates ocupó los puestos de presidente, director ejecutivo (CEO), presidente y arquitecto jefe de software. También fue el mayor accionista individual hasta mayo de 2014. Es uno de los empresarios más reconocidos y pioneros de la revolución de las microcomputadoras de los años setenta y ochenta.",
4: "La Mona Lisa es una obra de Leonardo.. Se celebra en el Louvre en París",
5: "Facebook es un servicio de red social lanzado como TheFacebook el 4 de febrero de 2004. Fue fundado por Mark Zuckerberg con sus compañeros de cuarto y compañeros de la Universidad de Harvard Eduardo Saverin, Andrew McCollum, Dustin Moskovitz y Chris Hughes. La membresía del sitio web estaba inicialmente limitada por los fundadores a los estudiantes de Harvard, pero se expandió a otras universidades en el área de Boston, la Ivy League y gradualmente a la mayoría de las universidades de los Estados Unidos y Canadá.",
6: "Geoffrey Everest Hinton es un psicólogo cognitivo e informático canadiense inglés, más conocido por su trabajo en redes neuronales artificiales. Desde 2013 divide su tiempo trabajando para Google y la Universidad de Toronto. En 2017, fue cofundador y se convirtió en el Asesor Científico Jefe del Vector Institute en Toronto.",
7: "Cuando le dije a John que quería mudarme a Alaska, me advirtió que tendría problemas para encontrar un Starbucks allí."
}

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

marian = MarianTransformer.pretrained("opus_mt_es_war", "xx")\
  .setInputCols(["sentence"])\
  .setOutputCol("translation")

marian_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])
light_pipeline = LightPipeline(marian_pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))

for sample in input_dict.values():
  result = light_pipeline.fullAnnotate(sample)
  print ('Translated:\n')
  for sentence in result[0]['translation']:
    print (sentence.result)
  print("-"*100)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_es_war download started this may take some time.
Approximate size to download 379.9 MB
[OK!]
Translated:

Gawas pa han pagin hadi ha norte, hi John Syy usa nga Ingles nga doktor ngan lider ha pag - uswag han tradisyonal nga mga medisina ngan kalimpyo ha mediko.
An siyahan nga paggamit hin impormasyon basi matambal an samad han kasina han 117.
----------------------------------------------------------------------------------------------------
Translated:

Hi James Haron, usa nga pelikula ha E.U.A., mahitungod han mga kalamidad ngan mga kalamidad, sinurat nga sinurat, ngan mga artikulo.
An makasaysayan ngan an komplikado nga mga bahin iginbasar ha mga istorya han Mga Ebanghelyo han Mga Ebanghelyo mahitungod han Mga Ebanghelyo.
Hi Leonardo Diits ngan hi Kasssssss mga membro han magkalainlain nga mga klase han sosiedad nga nahigugma ha barko durante han ira pagbiyahe
---------

### Translation Model from Thai to English

In [ ]:
data = "ประโยคของคุณที่จะแปล!"

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

marian = MarianTransformer.pretrained("opus_mt_th_en", "xx")\
  .setInputCols(["sentence"])\
  .setOutputCol("translation")

marian_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])
light_pipeline = LightPipeline(marian_pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))
result = light_pipeline.fullAnnotate(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_th_en download started this may take some time.
Approximate size to download 388.5 MB
[OK!]


### Translation Model from Turkish to English

In [ ]:
data = "Çevrilecek cümleniz!"

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

marian = MarianTransformer.pretrained("opus_mt_tr_en", "xx")\
  .setInputCols(["sentence"])\
  .setOutputCol("translation")

marian_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])
light_pipeline = LightPipeline(marian_pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))
result = light_pipeline.fullAnnotate(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_tr_en download started this may take some time.
Approximate size to download 389 MB
[OK!]


### Translation Model from Hungarian to English

In [ ]:
data = "Fordítandó mondatod!"

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

marian = MarianTransformer.pretrained("opus_mt_hu_en", "xx")\
  .setInputCols(["sentence"])\
  .setOutputCol("translation")

marian_pipeline = Pipeline(stages=[documentAssembler, sentencerDL, marian])
light_pipeline = LightPipeline(marian_pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))
result = light_pipeline.fullAnnotate(data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_hu_en download started this may take some time.
Approximate size to download 389.5 MB
[OK!]
